In [19]:
from pycaret.classification import *
#using knn undersampeling for neural net work

In [79]:
import pandas as pd
import numpy as np
dataset=pd.read_excel("C:/Users/97254/Downloads/patient_level__ver 12_proj1_sn_5-13.xlsx", engine='openpyxl')
print(dataset.shape)

(1020, 70)


In [80]:
dataset = dataset[~dataset['age'].isin(['NAN'])]

In [81]:
target = dataset[['ICU_mortality']]

target.head()

,ICU_mortality
0,0
1,0
2,0
3,0
4,0


In [82]:

dataset = dataset[['age','BMI','APACHEII','trauma','metabolic','gastrointestinal','Sepsis','EN_pro_day0/kg','EN_pro_day1/kg','EN_pro_day2/kg','EN_pro_day3/kg','EN_pro_day4/kg','EN_pro_day5/kg']]
print(dataset.shape)

(976, 13)


In [83]:
dataset.head()

,age,BMI,APACHEII,trauma,metabolic,gastrointestinal,Sepsis,EN_pro_day0/kg,EN_pro_day1/kg,EN_pro_day2/kg,EN_pro_day3/kg,EN_pro_day4/kg,EN_pro_day5/kg
0,72,23.875115,16,0,1,0,1,1.250297,1.221456,1.578462,0.995015,1.394308,1.403077
1,58,32.979592,23,0,1,0,1,0.758495,0.000000,0.670993,1.177788,0.403229,0.000000
2,44,22.857143,0,0,0,1,1,0.000000,0.000000,0.023259,1.396370,0.256457,0.000000
3,71,19.227688,0,0,1,1,0,1.145278,1.520000,1.520000,1.520000,1.520000,1.209667
4,60,32.791945,17,1,1,1,0,0.014902,0.858353,0.731984,0.922729,1.287529,1.287529


# under_sampling

In [84]:
from imblearn.under_sampling import EditedNearestNeighbours
  
#Edited Nearest Neighbours: This algorithm removes any sample which has labels different from those of its adjoining classes.
#pramaters to use:(ratio='auto', return_indices=False, random_state=None, size_ngh=None, n_neighbors=3, kind_sel='all', n_jobs=1)
en = EditedNearestNeighbours()
dataset, target = en.fit_resample(dataset, target)

In [85]:
dataset.shape

(589, 13)

In [86]:

downsampled = pd.concat([dataset, target],axis=1)

# checking counts
downsampled.ICU_mortality.value_counts()

0    354
1    235
Name: ICU_mortality, dtype: int64

In [87]:
downsampled.head()

,age,BMI,APACHEII,trauma,metabolic,gastrointestinal,Sepsis,EN_pro_day0/kg,EN_pro_day1/kg,EN_pro_day2/kg,EN_pro_day3/kg,EN_pro_day4/kg,EN_pro_day5/kg,ICU_mortality
0,58,32.979592,23,0,1,0,1,0.758495,0.000000,0.670993,1.177788,0.403229,0.000000,0
1,71,19.227688,0,0,1,1,0,1.145278,1.520000,1.520000,1.520000,1.520000,1.209667,0
2,57,34.717839,26,0,1,0,1,0.496533,0.275733,0.904000,0.811733,0.432533,0.000000,0
3,63,23.374726,16,0,0,0,1,0.785400,1.383591,1.582979,1.065138,1.327454,1.395000,0
4,39,18.730489,0,1,1,0,1,0.515885,0.922515,0.000000,0.000000,0.000000,0.000000,0


In [88]:
from sklearn.model_selection import train_test_split
#this function randomly split the data into train and test sets
downsampled = downsampled.astype('float32')
x_train, x_test, y_train, y_test = train_test_split(downsampled.iloc[:,:13] , downsampled.iloc[:,13], test_size=.2)

In [89]:
x_train.shape

(471, 13)

In [90]:
from keras.models import Sequential
from keras.layers import Dense
# load the dataset
# split into input (X) and output (y) variables
# define the keras model
model = Sequential()
model.add(Dense(20, input_dim=13, activation='relu'))
model.add(Dense(18, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(x_train, y_train, epochs=100, batch_size=20, verbose=0)
# make class predictions with the model

In [91]:
predictions = model.predict_classes(x_test)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[34, 27],
       [10, 47]], dtype=int64)

In [95]:
from keras.models import Sequential
from keras.layers import Dense
# load the dataset
# split into input (X) and output (y) variables
# define the keras model
model = Sequential()
model.add(Dense(20, input_dim=13, activation='relu'))
model.add(Dense(5, activation='sigmoid'))
model.add(Dense(10, activation='sigmoid'))

model.add(Dense(18, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(x_train, y_train, epochs=100, batch_size=20, verbose=0)
# make class predictions with the model

In [96]:
predictions = model.predict_classes(x_test)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[44, 17],
       [19, 38]], dtype=int64)